In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from zipfile import ZipFile
import os

# Specify the path to the zip file
zip_file_path = '/content/drive/MyDrive/audio.zip'

# Specify the extraction path
extract_path = '/content/drive/MyDrive'

# Unzip the file
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


os.listdir(extract_path)

['audio.zip', 'Colab Notebooks', 'audio']

In [3]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-tfj68wd3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-tfj68wd3
  Resolved https://github.com/huggingface/transformers to commit 0ad770c3733f9478a8d9d0bc18cc6143877b47a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8593793 sha256=ac83dc7d852ccf7842941d9f8fb60bc02f7ddab7547ca450667141a4bd3cb291
  Stored in directory: /tmp/pip-ephem-wheel-cache-ry_m4q8i/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.3 MB/s eta 0:00:00


**PROCESSING OF AUDIO DIRECTORY**

In [ ]:

import pandas as pd
from transformers import AutoTokenizer


def check_sequence_length(df, max_sequence_length=1024):
    for index, row in temp_df.iterrows():
        text = row['transcription']
        tokens = tokenizer.encode(text, add_special_tokens=True)

        if len(tokens) > max_sequence_length:
            print(f"Text at index {index} exceeds the max sequence length: {len(tokens)}")
            print(f"Text: {text}\n")

check_sequence_length(df)


In [ ]:

import os
import pandas as pd

audio_directory = '/content/drive/MyDrive/audio/data'
csv_files = set(df['file_name'])
all_audio_files = os.listdir(audio_directory)

files_to_remove = [file for file in all_audio_files if file not in csv_files]


for file_to_remove in files_to_remove:
    file_path = os.path.join(audio_directory, file_to_remove)
    os.remove(file_path)


**IMPORTING OPEN SOURCE WHISPER SMALL MODEL**

In [1]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [3]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

**CREATING AUDIO DATASET**

In [ ]:
import os
import shutil
import random

data_folder = "/content/drive/MyDrive/audio/data"

train_folder = os.path.join(data_folder, "train")
test_folder = os.path.join(data_folder, "test")

os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

audio_files = [f for f in os.listdir(data_folder) if f.endswith(".mp3")]

split_percentage = 0.8

num_train_files = int(len(audio_files) * split_percentage)
num_test_files = len(audio_files) - num_train_files

random.shuffle(audio_files)

for i, audio_file in enumerate(audio_files):
    source_path = os.path.join(data_folder, audio_file)
    if i < num_train_files:
        destination_path = os.path.join(train_folder, audio_file)
    else:
        destination_path = os.path.join(test_folder, audio_file)

    shutil.move(source_path, destination_path)

print(f"{num_train_files} files moved to 'train' folder.")
print(f"{num_test_files} files moved to 'test' folder.")

In [ ]:

import os
import pandas as pd

data_folder = "/content/drive/MyDrive/audio/data"

train_folder = os.path.join(data_folder, "train")
test_folder = os.path.join(data_folder, "test")

csv_file_path = "/content/drive/MyDrive/audio/data/metadata.csv"
df = pd.read_csv(csv_file_path)


def update_file_name(file_name, folder):
    return os.path.join(folder, os.path.basename(file_name))

for file_name in os.listdir(train_folder):
    if file_name.endswith(".mp3"):
        df.loc[df['file_name'].str.endswith(file_name), 'file_name'] = update_file_name(file_name, 'train')
for file_name in os.listdir(test_folder):
    if file_name.endswith(".mp3"):
        df.loc[df['file_name'].str.endswith(file_name), 'file_name'] = update_file_name(file_name, 'test')

df.to_csv(csv_file_path, index=False)
print(f"CSV file updated with file paths based on 'train' and 'test' folders.")

In [5]:
from datasets import load_dataset
dataset = load_dataset("audiofolder", data_dir="/content/drive/MyDrive/audio/data")


Resolving data files:   0%|          | 0/1690 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/423 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

**AUDIO DATASET PREPROCESSING**

In [7]:
def prepare_dataset(batch):
    # resample audio data from 48 to 16kHz
    audio = batch["audio"]

    batch["input_length"] = len(batch["audio"])

    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    batch["labels"] = tokenizer(batch["transcription"]).input_ids

    batch["labels_length"] = len(batch["labels"])
    return batch

In [8]:
MAX_DURATION_IN_SECONDS = 30.0
max_input_length = MAX_DURATION_IN_SECONDS * 16000


def filter_inputs(input_length):
    """Filter inputs with zero input length or longer than 30s"""
    return 0 < input_length < max_input_length

max_label_length = model.config.max_length

def filter_labels(labels_length):
    """Filter label sequences longer than max length (448)"""
    return labels_length < max_label_length

In [9]:
dataset = dataset.map(prepare_dataset, remove_columns= dataset.column_names["train"])

dataset = dataset.filter(filter_inputs, input_columns=["input_length"])

dataset = dataset.filter(filter_labels, input_columns=["labels_length"])

Map:   0%|          | 0/1690 [00:00<?, ? examples/s]

Map:   0%|          | 0/423 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1690 [00:00<?, ? examples/s]

Filter:   0%|          | 0/423 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1690 [00:00<?, ? examples/s]

Filter:   0%|          | 0/423 [00:00<?, ? examples/s]

In [11]:
 import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [12]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [13]:
import evaluate

metric = evaluate.load("wer")

In [15]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [14]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.generation_config.language = "en"


In [16]:
from huggingface_hub import notebook_login

notebook_login()

In [17]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-darpg",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=1000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=200,
    eval_steps=200,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [18]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [19]:
processor.save_pretrained(training_args.output_dir)

[]

In [20]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
200,0.902800,0.843902,90.673353
400,0.446000,0.556607,92.188956
600,0.178900,0.553851,86.706312
800,0.062500,0.640933,83.893218
1000,0.023100,0.716296,85.017337


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/do

TrainOutput(global_step=1000, training_loss=0.5397503607273102, metrics={'train_runtime': 8696.0709, 'train_samples_per_second': 1.84, 'train_steps_per_second': 0.115, 'total_flos': 4.5784073760768e+18, 'train_loss': 0.5397503607273102, 'epoch': 15.62})

**IMPORTING THE FINE TUNED MODEL**

In [6]:
from transformers import pipeline

pipe = pipeline(model="sanket003/whisper-darpg",
                batch_size=16,
                  chunk_length_s=30)  


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
